In [11]:
from __future__ import division
import math
import pygame
import collections
import itertools
from pyknon.music import NoteSeq, Note, Rest
from pyknon.genmidi import Midi
import os
from random import choice

In [2]:
def mod12(n):
    return n%12
def note_name(number): 
    notes = 'C C# D D# E F F# G G# A A# B'.split() 
    return notes[mod12(number)]
def accidentals(note_string):
    acc = len(note_string[1:])
    if '#' in note_string:
        return acc
    elif 'b' in note_string:
        return -acc
    else:
        return 0
def name_to_number(note_string):
    notes = 'C . D . E F . G . A . B'.split()
    name = note_string[0:1].upper()
    number = notes.index(name)
    acc = accidentals(note_string)
    return mod12(number + acc)
def durations(note_values, unity, tempo):
    return [note_duratino(nv, unity, tempo) for nv in notes_values]
def note_duration(note_value, unity, tempo):
    return (60.0 * note_value) / (tempo * unity)
def dotted_duration(duration, dots):
    ratio = Fraction(1,2)
    return duration * (1 - ratio**(dots+1))/ratio


In [3]:
def interval(x, y):
    return mod12(x-y)
def transposition(notes, index):
    return [mod12(n+index) for n in notes]
def retrograde(notes):
    return list(reversed(notes))
def rotate(item, n=1):
    modn = n % len(item)
    return item[modn:] + item[0:modn]
def inversion(notes, index=0):
    return [mod12(index - n) for n in notes]
# def inversion_startswith(notes, start):
#     transp = transposition_startswith(notes, 0)

In [4]:
def name_to_diatonic(note_string):
    notes = 'C D E F G A B'.split() 
    name = note_string[0:1].upper() 
    return notes.index(name)
def interval_name(note1, note2): 
    quantities = ['Unison', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh'] 
    n1, n2 = name_to_number(note1), name_to_number(note2) 
    d1, d2 = name_to_diatonic(note1), name_to_diatonic(note2) 
    chromatic_interval = interval(n2, n1) 
    diatonic_interval = (d2 - d1) % 7 
    quantity_name = quantities[diatonic_interval] 
    quality_name = get_quality(diatonic_interval, chromatic_interval) 
    return '%s %s' % (quality_name, quantity_name)
def get_quality(diatonic_interval, chromatic_interval): 
    if diatonic_interval in [0, 3, 4]:
        quality_map = ['Diminished', 'Perfect', 'Augmented'] 
    else: 
        quality_map = ['Diminished', 'Minor', 'Major', 'Augmented'] 
    index_map = [-1, 0, 2, 4, 6, 7, 9] 
    try: 
        return quality_map[chromatic_interval - index_map[diatonic_interval]] 
    except IndexError: 
        raise SimpleMusicError('Sorry, I can’t deal with this interval :-(')

In [5]:
def choice_if_list(item):
    if isinstance(item, collections.Iterable):
        return choice(item)
    else:
        return item
def random_notes(pitch_list, octave_list, duration, number_of_notes, volume=120):
    result = NoteSeq()
    for x in range(number_of_notes):
        pitch = choice(pitch_list)
        octave = choice_if_list(octave_list)
        dur = choice_if_list(duration)
        vol = choice_if_list(volume)
        result.append(Note(pitch, octave, dur, vol))
    return result
def random_notes_scale(root, majorMinor, octave_list, duration, number_of_notes, volume=120):
    root = name_to_number(root)
    pitch_list = [root]
    if majorMinor=="Major":
        interval = [2,2,1,2,2,2]
    else:
        interval = [2,1,2,2,1,2]
    for i in interval:
        pitch_list.append(root+i)
        root+=i
    print(pitch_list)
    return random_notes(pitch_list, octave_list, duration, number_of_notes, volume)

In [6]:
def fibonacci(n):
    result = []
    a, b = 0, 1
    while a < n:
        result.append(a)
        a, b = b, a + b
    return result
def pascals_triangle(n):
    x = [1]
    yield x
    for i in range(n - 1):
        x = [sum(i) for i in zip([0] + x, x + [0])]
        yield x
def play_list(pitch_list, octave_list, duration, volume=120):
    result = NoteSeq()
    for pitch in pitch_list:
        note = pitch%12
        octave = choice_if_list(octave_list)
        dur = choice_if_list(duration)
        vol = choice_if_list(volume)
        result.append(Note(note, octave, dur, vol))
    return result

In [7]:
def writeMidi(path, seq):
    if type(seq[0]) == Note:
        print("One Track")
        midi = Midi(1, tempo=100)
        midi.seq_notes(seq)
    else:
        midi = Midi(len(seq), tempo=100)
        i = 0
        for track in seq:
            midi.seq_notes(track, track=i)
            print(track)
            i+=1
    midi.write("test3.mid")
def playMidi(path):
    print("play")
    clock = pygame.time.Clock()
    pygame.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        clock.tick(60)
    print("end")
    os.remove(path)


In [14]:
seq = random_notes_scale('C#', 'Minor', range(5,6), [1/16,1/8, 1/4],50)
#seq = random_notes(range(0,12), range(5,7), [1/16,.125,.25],20)
seq
writeMidi('test3.mid', seq)
playMidi('test3.mid')

[1, 3, 4, 6, 8, 9, 11]
One Track
play
end


In [8]:
pattern = [0,2,4,5,7,9,11]
majorScales = {}
for i in range(12):
    majorScales[i] = [(j+i)%12 for j in pattern]
majorScales

{0: [0, 2, 4, 5, 7, 9, 11],
 1: [1, 3, 5, 6, 8, 10, 0],
 2: [2, 4, 6, 7, 9, 11, 1],
 3: [3, 5, 7, 8, 10, 0, 2],
 4: [4, 6, 8, 9, 11, 1, 3],
 5: [5, 7, 9, 10, 0, 2, 4],
 6: [6, 8, 10, 11, 1, 3, 5],
 7: [7, 9, 11, 0, 2, 4, 6],
 8: [8, 10, 0, 1, 3, 5, 7],
 9: [9, 11, 1, 2, 4, 6, 8],
 10: [10, 0, 2, 3, 5, 7, 9],
 11: [11, 1, 3, 4, 6, 8, 10]}

In [9]:
def getScales(notes):
    ans = []
    for scale in majorScales:
        if all(note in majorScales[scale] for note in notes):
            ans.append(note_name(scale))
    return ans
                

In [10]:
getScales([0,11])

['C', 'G']

In [18]:
def mod7(x):
    return x % 7
def getNoteNums(notes, scaleNum):
    scale = majorScales[scaleNum]
    ans = []
    for note in notes:
        for i in range(len(scale)):
            if note==scale[i]:
                ans.append(i)
    return ans
    #return [i for note in notes for i in range(len(scale)) if note==scale[i]]
def getNotesFromNums(nums, scaleNum):
    scale = majorScales[scaleNum]
    ans = []
    for num in nums:
        for i in range(len(scale)):
            if num==i:
                ans.append(scale[i])
    return ans
    #return [scale[i] for i in range(len(scale)) for num in nums if num==i]
#invert
def scaleInvert(notes, scaleNum, index=6):
    scale = majorScales[scaleNum]
    NoteNums = getNoteNums(notes, scaleNum)
    inverted = [mod7(index - n) for n in NoteNums]
    return getNotesFromNums(inverted, scaleNum)
#transpose
def scaleTranspose(notes, scaleNum, distance=0):
    scale = majorScales[scaleNum]
    NoteNums = getNoteNums(notes, scaleNum)
    transposed = [mod7(n + distance) for n in NoteNums]
    return getNotesFromNums(transposed, scaleNum)
    pass
#stretch_Duration
def scaleStretchDuration(notes, factor):
    return notes.stretch_dur(factor)
#stretch_interval
def scaleStretchInterval(notes, scaleNum, factor):
    scale = majorScales[scaleNum]
    NoteNums = getNoteNums(notes, scaleNum)
    intervals = [NoteNums[i+1] - NoteNums[i] for i in range(len(NoteNums)-1)]
    print(intervals)
    stretched = [math.ceil(n+factor) for n in intervals]
    print(stretched)
    resultNums = [NoteNums[0]]
    for i in stretched:
        resultNums.append(resultNums[-1]+i)
    return getNotesFromNums(resultNums, scaleNum)
def rotation(notes, degree):
    return notes.rotation(degree)

In [19]:
notes = [0,2,4]
scaleStretchInterval(notes, 0, 1)

[1, 1]
[2, 2]


[0, 4, 7]

In [139]:
def abstraction():
    a = NoteSeq("C4. Eb8")
    a1 = a.transp(5).stretch_dur(0.5)
    a2 = a.inv("Db''")
    a3 = a1.inv(8)
    
    A = a + a1 + a2 + a3
    A2 = A.transp(2)
    B = a1.transp(8) + a1.transp("Eb''")
    
    c = NoteSeq([Note(x.value, dur=0.125) for x in a+a1])
    
    C = (c.inv("Ab''") +
        c.inv(10) + 
        c.stretch_interval(2).transp(2) + 
        c.inv("G''") + 
        c.inv("E''").stretch_interval(1) + 
        c.inv("A").stretch_interval(1)
        )
    a4 = a.stretch_dur(2).inv(6)
    
    Part1 = A + NoteSeq("C2") + A2 + B 
    Part2 = C + a4
    writeMidi('test3.mid', Part1 + Part2)
    playMidi('test3.mid')
abstraction()

One Track
play
end
